# pygrass Modules, advance usage

We already pygrass `Modules` module in the unit 02. Now we will see other modules to run `Modules` in different ways. 

In [ ]:
from IPython.display import Image
# pretty-print
from pprint import pprint
from subprocess import PIPE
from multiprocessing import Process

## MultiModules, run several modules in serial

`MultiModules` is designed to run a list of modules in serial in the provided order within a temporary region environment. `Modules` can be run in serial synchronously or asynchronously.

In [ ]:
from grass.pygrass.modules import Module
from grass.pygrass.modules import MultiModule

Let's create to Modules objects and try to run them with `MultiModules`

In [ ]:
# we need to remember to use run_=False
listvect = Module('g.list', type='vector', mapset='.', stdout_=PIPE, run_=False)
listraster = Module('g.list', type='raster', mapset='.', stdout_=PIPE, run_=False)

In [ ]:
# we can i
mm = MultiModule(module_list=[listvect, listraster])
mm.run()

At this point you can see the output of the modules

In [ ]:
print(mm.module_list[0].outputs.stdout)

In [ ]:
print(mm.module_list[1].outputs.stdout)

It is possible to get the help for `MultiModules`

In [ ]:
mm?

It is possible to test the asynchronous mode.
You can test it running several map algebra with `r.mapcalc` module. First set the right Region and get info about the raster

In [ ]:
print(Module('g.region', raster='elevation@PERMANENT', flags='p', stdout_=PIPE).outputs.stdout)
print(Module('r.info', map='elevation@PERMANENT', flags='r', stdout_=PIPE).outputs.stdout)

Now it is possible to create a list of `r.mapcalc` `Modules` objects. 

In [ ]:
listmapcalc = []
val = 0
while val < 200:
    listmapcalc.append(Module("r.mapcalc",
                       expression="elevation_{va}=if(elevation@PERMANENT>{va} && elevation@PERMANENT<={va2}, elevation@PERMANENT, null())".format(va=val, va2=val+50),
                       overwrite=True, run_=False, finish_=False)
    )
    val += 50


In [ ]:
listmapcalc[-1].get_bash()

At this point create the `MultiModules` object in asynchronous mode and run it.

In [ ]:
mm = MultiModule(module_list=listmapcalc, sync=False)
pro = mm.run()
isinstance(pro, Process)

In [ ]:
mapcalc_list = mm.wait()
for mapcalc in mapcalc_list:
    print(mapcalc.popen.returncode)
print(Module('g.list', type='raster', pattern='elevation*', stdout_=PIPE).outputs.stdout)

## GridModules, run a single process in multiprocessing

`GridModules` provide a simple Module to run a GRASS process and split analysis between several processes. 

In [ ]:
from grass.pygrass.modules.grid.grid import GridModule

In [ ]:
# create GridModule object and get the different works
grd = GridModule('r.slope.aspect',
                 width=1000, height=1000, overlap=2,
                 processes=None, split=False,
                 elevation='elevation@PERMANENT',
                 slope='slope_grid', aspect='aspect_grid', overwrite=True)
grd.get_works()

At this point run it and check if the maps are created...

In [ ]:
grd.run()

In [ ]:
print(Module('g.list', type='raster', mapset='.', stdout_=PIPE, pattern="*_grid").outputs.stdout)

# Summary

We have seen

* how to run several command in serial mode
* how to run a command splitting its calculation in several process. 
  **Pay attention because for several analysis this is no possible**

# Exercise

## Exercise 1

Computes the viewshed for ten point on the `elevation` raster map inside the computational region

## Exercise 2

Run `r.geomorphon` as single process with `Module` and using `GridModule` and check if the results are the same